<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D9%85%D8%A7%D9%83%D8%AF111.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    """حساب مؤشر MACD"""
    data['EMA_short'] = data['close'].ewm(span=short_window, adjust=False).mean()
    data['EMA_long'] = data['close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data

# استخدام الدالة
data = calculate_macd(data)
ذي

In [ ]:
def calculate_bollinger_bands(data, window=20, num_std_dev=2):
    """حساب بولينجر باندز."""
    try:
        # حساب المتوسط المتحرك
        data['Middle_Band'] = data['close'].rolling(window=window).mean()

        # حساب الانحراف المعياري
        data['Std_Dev'] = data['close'].rolling(window=window).std()

        # حساب الحد العلوي والحد السفلي
        data['Upper_Band'] = data['Middle_Band'] + (data['Std_Dev'] * num_std_dev)
        data['Lower_Band'] = data['Middle_Band'] - (data['Std_Dev'] * num_std_dev)

        return data
    except Exception as e:
        print(f"An error occurred while calculating Bollinger Bands: {e}")
        return data


In [ ]:
data = get_and_clean_gold_prices()
if data is not None:
    data = calculate_bollinger_bands(data)  # إضافة هذه السطر لحساب بولينجر باندز
    build_strategy(data)


In [ ]:
def calculate_stochastic(data, k_window=14, d_window=3):
    """حساب Stochastic Oscillator"""
    low_min = data['low'].rolling(window=k_window).min()
    high_max = data['high'].rolling(window=k_window).max()
    data['%K'] = 100 * (data['close'] - low_min) / (high_max - low_min)
    data['%D'] = data['%K'].rolling(window=d_window).mean()
    return data


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp

            distance_to_tp = take_profit - entry_price

            current_price = mt5.symbol_info_tick(symbol).bid

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.72 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)
                update_stop_loss(position, new_stop_loss)


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            distance_to_tp = take_profit - entry_price

            current_price = mt5.symbol_info_tick(symbol).bid

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.72 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if new_stop_loss > current_stop_loss:
                update_stop_loss(position, new_stop_loss)

In [ ]:
import pandas as pd

# تعريف دالة حساب ADX
def calculate_adx(data, period=14):
    """حساب مؤشر ADX."""
    data['High_Change'] = data['high'].diff()
    data['Low_Change'] = data['low'].diff()

    data['+DM'] = (data['High_Change'] > data['Low_Change']) & (data['High_Change'] > 0) * data['High_Change']
    data['-DM'] = (data['Low_Change'] > data['High_Change']) & (data['Low_Change'] > 0) * data['Low_Change']

    data['TR'] = data[['high', 'low', 'close']].max(axis=1) - data[['high', 'low', 'close']].min(axis=1)
    data['ATR'] = data['TR'].rolling(window=period).mean()

    data['+DI'] = 100 * (data['+DM'].rolling(window=period).sum() / data['ATR'])
    data['-DI'] = 100 * (data['-DM'].rolling(window=period).sum() / data['ATR'])

    data['DX'] = 100 * (abs(data['+DI'] - data['-DI']) / (data['+DI'] + data['-DI']))
    data['ADX'] = data['DX'].rolling(window=period).mean()

    return data[['ADX']]

# استخدم الدالة مع بياناتك الخاصة
# تأكد من أن لديك DataFrame بالبيانات المطلوبة (high, low, close)
# adx_data = calculate_adx(your_dataframe)

# طباعة النتائج
# print(adx_data)


In [ ]:
import time

def check_for_high_impact_news():
    """تحقق من وجود أخبار قوية قادمة."""
    # يمكنك استخدام مكتبة مثل requests لجلب الأخبار من موقع مثل Investing.com
    # أو يمكنك استخدام API إذا كان متاحًا.
    # هنا مثال بسيط:
    url = "https://www.investing.com/economic-calendar/"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # ابحث عن الأخبار ذات التأثير العالي
    high_impact_news = soup.select('div.high-impact-news')  # هذا افتراض
    return len(high_impact_news) > 0

# تنفيذ الاستراتيجية بشكل مستمر
if __name__ == "__main__":
    while True:
        data = get_and_clean_gold_prices()
        if data is not None:
            build_strategy(data)

        # تحقق من وجود أخبار قوية
        if check_for_high_impact_news():
            time.sleep(30)  # الانتظار لمدة 30 ثانية قبل التكرار
        else:
            time.sleep(60)  # الانتظار لمدة دقيقة قبل التكرار


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            distance_to_tp = take_profit - entry_price

            current_price = mt5.symbol_info_tick(symbol).bid

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if new_stop_loss > current_stop_loss:
                update_stop_loss(position, new_stop_loss)


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على أقرب سعر وصل إليه السعر نحو الهدف بعد سعر افتتاح الصفقة."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # الحصول على أسعار التاريخية بعد فتح الصفقة
            historical_prices = get_historical_prices(symbol, entry_price)

            # تحديد أقرب سعر وصل إليه السعر نحو تيك بروفت
            closest_price = max([price for price in historical_prices if price >= entry_price], default=entry_price)

            distance_to_tp = take_profit - entry_price

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if closest_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)

            if closest_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)

            if closest_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if new_stop_loss > current_stop_loss:
                update_stop_loss(position, new_stop_loss)

def get_historical_prices(symbol, entry_price):
    """استرجاع الأسعار التاريخية بعد سعر افتتاح الصفقة."""
    # يمكنك تعديل هذه الدالة لتناسب احتياجاتك، مثل استخدام `copy_rates_from_pos`
    rates = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M15, 0, 1000)
    prices = [rate['close'] for rate in rates if rate['time'] >= entry_price]
    return prices


In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_momentum(data)  # حساب الزخم
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # تحقق من عدد الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol="XAUUSD")
        open_positions_count = len(open_positions)

        # إذا كان عدد الصفقات المفتوحة 7، لا تفتح أي صفقات جديدة
        if open_positions_count >= 7:
            print("عدد الصفقات المفتوحة بلغ الحد الأقصى (7). لا يمكن فتح صفقات جديدة.")
            return

        # تحديد الاتجاه العام باستخدام المتوسط المتحرك
        current_ma = data['MA'].iloc[-1]
        previous_ma = data['MA'].iloc[-2]

        # تحقق مما إذا كان الاتجاه قد تغير
        new_trend = "bullish" if current_ma > previous_ma else "bearish"

        current_positions = "buy" if open_positions and open_positions[0].type == 0 else "sell"

        # إذا كان الاتجاه قد تغير، أغلق الصفقات الحالية وافتح صفقات جديدة
        if (new_trend == "bullish" and current_positions == "sell") or (
                new_trend == "bearish" and current_positions == "buy"):
            # إغلاق الصفقات المفتوحة
            for position in open_positions:
                request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "position": position.ticket,
                    "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                    "volume": position.volume,
                    "price": mt5.symbol_info_tick("XAUUSD").bid if position.type == 0 else mt5.symbol_info_tick(
                        "XAUUSD").ask,
                    "deviation": 10,
                    "magic": 234000,
                    "comment": "Closing position..."
                }
                result = mt5.order_send(request)

                if result is None or (hasattr(result, 'retcode') and result.retcode != mt5.TRADE_RETCODE_DONE):
                    print(f"فشل في إغلاق الصفقة: {result.retcode if result else 'Unknown error'}")
                else:
                    print(f"تم إغلاق الصفقة بنجاح: {position.ticket}")

        # فتح صفقة جديدة بناءً على الاتجاه الجديد
        if open_positions_count < 7:  # تحقق من العدد قبل فتح صفقة جديدة
            order_type = "buy" if new_trend == "bullish" else "sell"
            volume = calculate_lot_size(account_balance=mt5.account_info().balance)
            print(f"Calculated volume: {volume}")

            price = mt5.symbol_info_tick("XAUUSD").ask if order_type == "buy" else mt5.symbol_info_tick("XAUUSD").bid

            # تعيين SL و TP
            sl = price - 10 if order_type == "buy" else price + 10
            tp = price + 20 if order_type == "buy" else price - 20

            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": "XAUUSD",
                "volume": volume,
                "type": 0 if order_type == "buy" else 1,  # 0 للشراء و 1 للبيع
                "price": price,
                "sl": sl,
                "tp": tp,
                "deviation": 10,
                "magic": 234000,
                "comment": "Opening new position..."
            }

            result = mt5.order_send(request)

            if result is None or (hasattr(result, 'retcode') and result.retcode != mt5.TRADE_RETCODE_DONE):
                print(f"فشل في فتح الصفقة: {result.retcode if result else 'Unknown error'}")
            else:
                print(f"تم فتح الصفقة بنجاح: {order_type} {volume} lots من XAUUSD عند {price}")

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب نقاط وولف ويف
        wolf_wave_points = calculate_wolf_wave(data)
        calculate_stochastic(data)
        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # استخدام تحليل المشاعر لتعديل استراتيجيتك
        if average_sentiment > 0.1 and open_positions_count < 7:
            place_order("XAUUSD", "buy", volume=0.01)
        elif average_sentiment < -0.1 and open_positions_count < 7:
            place_order("XAUUSD", "sell", volume=0.01)

        # تأمين الصفقة إذا كانت هناك خسارة كبيرة
        if check_open_positions("XAUUSD"):
            hedge_position_with_retrace_limit("XAUUSD")  # تأمين الصفقة
            set_break_even("XAUUSD")  # تعيين بريك إيفن

            # إغلاق الصفقات الرابحة إذا كانت مجموع الأرباح تعادل ثلاث أرباع مجموع الخسائر
            positions = mt5.positions_get(symbol="XAUUSD")
            total_profit = sum(position.profit for position in positions if position.profit > 0)
            total_loss = sum(-position.profit for position in positions if position.profit < 0)

            # تحقق من الشرط
            if total_profit >= (3 / 4) * total_loss and total_loss > 0:  # تأكد من وجود خسائر
                for position in positions:
                    if position.profit > 0:  # إغلاق الصفقة الرابحة
                        request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "position": position.ticket,
                            "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                            "volume": position.volume,
                            "price": mt5.symbol_info_tick("XAUUSD").bid if position.type == 0 else mt5.symbol_info_tick(
                                "XAUUSD").ask,
                            "deviation": 10,
                            "magic": 234000,
                            "comment": "Closing profitable position",
                            "type_time": mt5.ORDER_TIME_GTC,
                            "type_filling": mt5.ORDER_FILLING_IOC,
                        }
                        result = mt5.order_send(request)
                        if result.retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"تم إغلاق الصفقة الرابحة لـ {position.ticket}.")

        # إنشاء ميزات إضافية
        data = create_features(data)  # إنشاء ميزات إضافية
        data = calculate_price_channel(data)  # حساب قناة السعر
        data = advanced_risk_management(data)  # إدارة المخاطر المتقدمة

        # حساب مناطق الأوامر
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على التنبؤ
        last_row = data.iloc[-1]
        predicted_price = model_xgb.predict(data.drop(['target'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        if open_positions_count < 7:  # تحقق من عدم وجود صفقات مفتوحة
            if predicted_price > last_row['close']:
                place_order("XAUUSD", "buy", volume=0.01)
            elif predicted_price < last_row['close']:
                place_order("XAUUSD", "sell", volume=0.01)

        # رسم مناطق الطلب والعرض
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""

    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # تحديد السعر الحالي بناءً على نوع الصفقة
            if position.type == mt5.ORDER_BUY:
                current_price = mt5.symbol_info_tick(symbol).bid
                distance_to_tp = take_profit - entry_price
            elif position.type == mt5.ORDER_SELL:
                current_price = mt5.symbol_info_tick(symbol).ask
                distance_to_tp = entry_price - take_profit

            # حساب الأهداف المختلفة
            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp) if position.type == mt5.ORDER_BUY else entry_price - (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp) if position.type == mt5.ORDER_BUY else entry_price - (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp) if position.type == mt5.ORDER_BUY else entry_price - (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if (position.type == mt5.ORDER_BUY and new_stop_loss > current_stop_loss) or \
               (position.type == mt5.ORDER_SELL and new_stop_loss < current_stop_loss):
                update_stop_loss(position, new_stop_loss)


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # تحديد السعر الحالي بناءً على نوع الصفقة
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # حساب المسافة إلى جني الأرباح
            distance_to_tp = take_profit - entry_price if position.type == 0 else entry_price - take_profit

            # حساب الأهداف المختلفة
            fifty_percent_target = entry_price + (0.36 * distance_to_tp) if position.type == 0 else entry_price - (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp) if position.type == 0 else entry_price - (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp) if position.type == 0 else entry_price - (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            # تحديث وقف الخسارة بناءً على السعر الحالي
            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp) if position.type == 0 else entry_price - (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp) if position.type == 0 else entry_price - (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp) if position.type == 0 else entry_price - (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if (position.type == 0 and new_stop_loss > current_stop_loss) or \
               (position.type == 1 and new_stop_loss < current_stop_loss):
                update_stop_loss(position, new_stop_loss)


In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة7777 بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # تحديد السعر الحالي بناءً على نوع الصفقة
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # حساب المسافة إلى جني الأرباح (مع مراعاة نوع الصفقة)
            distance_to_tp = take_profit - entry_price if position.type == 0 else entry_price - take_profit

            # حساب الأهداف المختلفة (مع مراعاة نوع الصفقة)
            fifty_percent_target = entry_price + (0.36 * distance_to_tp) if position.type == 0 else entry_price - (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp) if position.type == 0 else entry_price - (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp) if position.type == 0 else entry_price - (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            # تحديث وقف الخسارة بناءً على السعر الحالي (مع مراعاة نوع الصفقة)
            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp) if position.type == 0 else entry_price - (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp) if position.type == 0 else entry_price - (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp) if position.type == 0 else entry_price - (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم (مع مراعاة نوع الصفقة)
            if (position.type == 0 and new_stop_loss > current_stop_loss) or \
               (position.type == 1 and new_stop_loss < current_stop_loss):
                update_stop_loss(position, new_stop_loss)

In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # تحديد السعر الحالي بناءً على نوع الصفقة
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # حساب المسافة إلى جني الأرباح (مع مراعاة نوع الصفقة)
            distance_to_tp = abs(take_profit - entry_price)  # استخدام المسافة المطلقة دائمًا

            # حساب الأهداف المختلفة (مع مراعاة نوع الصفقة)
            fifty_percent_target = entry_price + (0.36 * distance_to_tp) if position.type == 0 else entry_price - (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp) if position.type == 0 else entry_price - (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp) if position.type == 0 else entry_price - (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            # تحديث وقف الخسارة بناءً على السعر الحالي (مع مراعاة نوع الصفقة)
            if current_price >= fifty_percent_target and current_price < seventy_five_percent_target:  # إضافة فحص النطاق
                new_stop_loss = entry_price + (0.12 * distance_to_tp) if position.type == 0 else entry_price - (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target and current_price < ninety_percent_target:  # إضافة فحص النطاق
                new_stop_loss = entry_price + (0.24 * distance_to_tp) if position.type == 0 else entry_price - (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp) if position.type == 0 else entry_price - (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم (مع مراعاة نوع الصفقة)
            if (position.type == 0 and new_stop_loss > current_stop_loss) or \
               (position.type == 1 and new_stop_loss < current_stop_loss):
                # فرض تحديث إيقاف الخسارة حتى لو كان التغيير صغيرًا جدًا
                if abs(new_stop_loss - current_stop_loss) > mt5.symbol_info(symbol).trade_tick_size:
                    update_stop_loss(position, new_stop_loss)

In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف،
    مع عدم تحريك الستوب لوز إذا كان الجديد أقل من القديم."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            distance_to_tp = take_profit - entry_price

            current_price = mt5.symbol_info_tick(symbol).bid

            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if new_stop_loss > current_stop_loss:
                update_stop_loss(position, new_stop_loss)



In [ ]:
def update_stop_loss(position, new_stop_loss):
    """تحديث وقف الخسارة للصفقة المحددة مع الحفاظ على تيك بروفت."""
    request = {
        "action": mt5.TRADE_ACTION_SLTP,
        "position": position.ticket,
        "sl": new_stop_loss,
        "tp": position.tp,  # الحفاظ على تيك بروفت كما هو
        "symbol": position.symbol,
        "magic": position.magic,
        "deviation": 10,
        "comment": "Update Stop Loss",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    return result


def set_break_even():
    """تعيين وقف الخسارة إلى مستويات مختلفة لجميع الصفقات المفتوحة، مع عدم التراجع."""
    positions = mt5.positions_get()

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp  # سعر جني الأرباح

            # حساب المسافة بين سعر الدخول وسعر جني الأرباح
            distance_to_tp = take_profit - entry_price

            # الحصول على السعر الحالي
            current_price = mt5.symbol_info_tick(position.symbol).bid  # الحصول على السعر الحالي

            # حساب النسب المئوية
            fifty_percent_target = entry_price + (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.72 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp)

            # تعيين وقف الخسارة بناءً على تقدم السعر
            new_stop_loss = position.sl  # الحصول على وقف الخسارة الحالي

            if current_price >= fifty_percent_target:
                new_stop_loss = max(new_stop_loss, entry_price + (0.12 * distance_to_tp))
                update_stop_loss(position, new_stop_loss)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = max(new_stop_loss, entry_price + (0.24 * distance_to_tp))
                update_stop_loss(position, new_stop_loss)

            if current_price >= ninety_percent_target:
                new_stop_loss = max(new_stop_loss, entry_price + (0.48 * distance_to_tp))
                update_stop_loss(position, new_stop_loss)

In [ ]:
def set_break_even(symbol):
    """تعيين وقف الخسارة إلى مستويات مختلفة بناءً على تقدم السعر نحو الهدف لكل صفقة."""
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl  # الحصول على قيمة الستوب لوز الحالية

            # تحديد السعر الحالي بناءً على نوع الصفقة
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # حساب المسافة إلى جني الأرباح
            distance_to_tp = take_profit - entry_price if position.type == 0 else entry_price - take_profit

            # حساب الأهداف المختلفة
            fifty_percent_target = entry_price + (0.36 * distance_to_tp) if position.type == 0 else entry_price - (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp) if position.type == 0 else entry_price - (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp) if position.type == 0 else entry_price - (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss  # افتراض أن الستوب لوز لن يتغير

            # تحديث وقف الخسارة بناءً على السعر الحالي
            if current_price >= fifty_percent_target:
                new_stop_loss = entry_price + (0.12 * distance_to_tp) if position.type == 0 else entry_price - (0.12 * distance_to_tp)

            if current_price >= seventy_five_percent_target:
                new_stop_loss = entry_price + (0.24 * distance_to_tp) if position.type == 0 else entry_price - (0.24 * distance_to_tp)

            if current_price >= ninety_percent_target:
                new_stop_loss = entry_price + (0.48 * distance_to_tp) if position.type == 0 else entry_price - (0.48 * distance_to_tp)

            # مقارنة الستوب لوز الجديد بالقديم
            if (position.type == 0 and new_stop_loss > current_stop_loss) or \
               (position.type == 1 and new_stop_loss < current_stop_loss):
                update_stop_loss(position, new_stop_loss)


In [ ]:
# prompt: انشئ دالة بريك ايفن تعمل على جميع الصفقات بيع او شراء و في نفس اللحظة

def set_break_even(symbol):
    """
    Sets the stop-loss to different levels based on price progress towards the target,
    without moving the stop-loss if the new one is lower than the old one.
    Works for both buy and sell orders simultaneously.
    """
    positions = mt5.positions_get(symbol=symbol)

    if positions:
        for position in positions:
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl

            # Determine the current price based on the order type
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # Calculate the distance to take profit (absolute value)
            distance_to_tp = abs(take_profit - entry_price)

            # Calculate targets (adjust for buy/sell)
            fifty_percent_target = entry_price + (0.36 * distance_to_tp) if position.type == 0 else entry_price - (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp) if position.type == 0 else entry_price - (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp) if position.type == 0 else entry_price - (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss

            # Update stop-loss based on current price and order type
            if position.type == 0 and current_price >= fifty_percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.12 * distance_to_tp))
            elif position.type == 1 and current_price <= fifty_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.12 * distance_to_tp))


            if position.type == 0 and current_price >= seventy_five_percent_target :  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.24 * distance_to_tp))
            elif position.type == 1 and current_price <= seventy_five_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.24 * distance_to_tp))


            if position.type == 0 and current_price >= ninety_percent_target: #Buy Order
                new_stop_loss = max(new_stop_loss, entry_price + (0.48 * distance_to_tp))
            elif position.type == 1 and current_price <= ninety_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.48 * distance_to_tp))


            # Check if the new stop-loss is different and valid before updating
            if abs(new_stop_loss - current_stop_loss) > mt5.symbol_info(symbol).trade_tick_size:
                update_stop_loss(position, new_stop_loss)

In [ ]:
def place_order(symbol, order_type, volume):
    """فتح صفقة في MetaTrader 5 مع حجم ديناميكي."""
    open_positions_count = len(mt5.positions_get(symbol=symbol))

    # لا تفتح صفقة جديدة إذا كان عدد الصفقات 7 أو أكثر
    if open_positions_count >= 7:
        print("لا يمكن فتح صفقة جديدة. الحد الأقصى للصفقات المفتوحة (7) تم الوصول إليه.")
    # فتح صفقة جديدة إذا كان عدد الصفقات 4 أو أقل
    elif open_positions_count <= 4:
        # كود لفتح الصفقة
        print(f"فتح صفقة جديدة: {order_type} {volume} lots من {symbol}.")
    else:
        print("عدد الصفقات المفتوحة لا يسمح بفتح صفقة جديدة.")


In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_momentum(data)  # حساب الزخم
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # تحقق من عدد الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol="XAUUSD")
        open_positions_count = len(open_positions)

        # إذا كان عدد الصفقات المفتوحة 7، لا تفتح أي صفقات جديدة
        if open_positions_count >= 7:
            print("عدد الصفقات المفتوحة بلغ الحد الأقصى (7). لا يمكن فتح صفقات جديدة.")
            return

        # تحديد الاتجاه العام باستخدام المتوسط المتحرك
        current_ma = data['MA'].iloc[-1]
        previous_ma = data['MA'].iloc[-2]

        # تحقق مما إذا كان الاتجاه قد تغير
        new_trend = "bullish" if current_ma > previous_ma else "bearish"

        current_positions = "buy" if open_positions and open_positions[0].type == 0 else "sell"

        # إذا كان الاتجاه قد تغير، أغلق الصفقات الحالية وافتح صفقات جديدة
        if (new_trend == "bullish" and current_positions == "sell") or (
                new_trend == "bearish" and current_positions == "buy"):
            # إغلاق الصفقات المفتوحة
            for position in open_positions:
                request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "position": position.ticket,
                    "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                    "volume": position.volume,
                    "price": mt5.symbol_info_tick("XAUUSD").bid if position.type == 0 else mt5.symbol_info_tick(
                        "XAUUSD").ask,
                    "deviation": 10,
                    "magic": 234000,
                    "comment": "Closing position..."
                }
                result = mt5.order_send(request)

                if result is None or (hasattr(result, 'retcode') and result.retcode != mt5.TRADE_RETCODE_DONE):
                    print(f"فشل في إغلاق الصفقة: {result.retcode if result else 'Unknown error'}")
                else:
                    print(f"تم إغلاق الصفقة بنجاح: {position.ticket}")

        # فتح صفقة جديدة بناءً على الاتجاه الجديد إذا كان عدد الصفقات المفتوحة أقل من 4
        if open_positions_count < 4:  # تحقق من العدد قبل فتح صفقة جديدة
            order_type = "buy" if new_trend == "bullish" else "sell"
            volume = calculate_lot_size(account_balance=mt5.account_info().balance)
            print(f"Calculated volume: {volume}")

            # استدعاء دالة place_order
            place_order(symbol="XAUUSD", order_type=order_type, volume=volume)

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب نقاط وولف ويف
        wolf_wave_points = calculate_wolf_wave(data)
        calculate_stochastic(data)
        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # استخدام تحليل المشاعر لتعديل استراتيجيتك
        if average_sentiment > 0.1 and open_positions_count < 4:
            place_order("XAUUSD", "buy", volume=0.01)
        elif average_sentiment < -0.1 and open_positions_count < 4:
            place_order("XAUUSD", "sell", volume=0.01)

        # تأمين الصفقة إذا كانت هناك خسارة كبيرة
        if check_open_positions("XAUUSD"):
            hedge_position_with_retrace_limit("XAUUSD")  # تأمين الصفقة
            set_break_even("XAUUSD")  # تعيين بريك إيفن

            # إغلاق الصفقات الرابحة إذا كانت مجموع الأرباح تعادل ثلاث أرباع مجموع الخسائر
            positions = mt5.positions_get(symbol="XAUUSD")
            total_profit = sum(position.profit for position in positions if position.profit > 0)
            total_loss = sum(-position.profit for position in positions if position.profit < 0)

            # تحقق من الشرط
            if total_profit >= (3 / 4) * total_loss and total_loss > 0:  # تأكد من وجود خسائر
                for position in positions:
                    if position.profit > 0:  # إغلاق الصفقة الرابحة
                        request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "position": position.ticket,
                            "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                            "volume": position.volume,
                            "price": mt5.symbol_info_tick("XAUUSD").bid if position.type == 0 else mt5.symbol_info_tick(
                                "XAUUSD").ask,
                            "deviation": 10,
                            "magic": 234000,
                            "comment": "Closing profitable position",
                            "type_time": mt5.ORDER_TIME_GTC,
                            "type_filling": mt5.ORDER_FILLING_IOC,
                        }
                        result = mt5.order_send(request)
                        if result.retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"تم إغلاق الصفقة الرابحة لـ {position.ticket}.")

        # إنشاء ميزات إضافية
        data = create_features(data)  # إنشاء ميزات إضافية
        data = calculate_price_channel(data)  # حساب قناة السعر
        data = advanced_risk_management(data)  # إدارة المخاطر المتقدمة

        # حساب مناطق الأوامر
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على التنبؤ
        last_row = data.iloc[-1]
        predicted_price = model_xgb.predict(data.drop(['target'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        if open_positions_count < 4:  # تحقق من عدم وجود صفقات مفتوحة
            if predicted_price > last_row['close']:
                place_order("XAUUSD", "buy", volume=0.01)
            elif predicted_price < last_row['close']:
                place_order("XAUUSD", "sell", volume=0.01)

        # رسم مناطق الطلب والعرض
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_momentum(data)  # حساب الزخم
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # تحقق من عدد الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol="XAUUSD")
        open_positions_count = len(open_positions)

        # إذا كان عدد الصفقات المفتوحة 7، لا تفتح أي صفقات جديدة
        if open_positions_count >= 7:
            print("عدد الصفقات المفتوحة بلغ الحد الأقصى (7). لا يمكن فتح صفقات جديدة.")
            return

        # تحديد الاتجاه العام باستخدام المتوسط المتحرك
        current_ma = data['MA'].iloc[-1]
        previous_ma = data['MA'].iloc[-2]

        # تحقق مما إذا كان الاتجاه قد تغير
        new_trend = "bullish" if current_ma > previous_ma else "bearish"

        current_positions = "buy" if open_positions and open_positions[0].type == 0 else "sell"

        # إذا كان الاتجاه قد تغير، أغلق الصفقات الحالية وافتح صفقات جديدة
        if (new_trend == "bullish" and current_positions == "sell") or (
                new_trend == "bearish" and current_positions == "buy"):
            # إغلاق الصفقات المفتوحة
            for position in open_positions:
                request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "position": position.ticket,
                    "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                    "volume": position.volume,
                    "price": mt5.symbol_info_tick("XAUUSD").bid if position.type == 0 else mt5.symbol_info_tick(
                        "XAUUSD").ask,
                    "deviation": 10,
                    "magic": 234000,
                    "comment": "Closing position..."
                }
                result = mt5.order_send(request)

                if result is None or (hasattr(result, 'retcode') and result.retcode != mt5.TRADE_RETCODE_DONE):
                    print(f"فشل في إغلاق الصفقة: {result.retcode if result else 'Unknown error'}")
                else:
                    print(f"تم إغلاق الصفقة بنجاح: {position.ticket}")

        # فتح صفقة جديدة بناءً على الاتجاه الجديد إذا كان العدد أقل من 5
        if open_positions_count < 5:  # تحقق من العدد قبل فتح صفقة جديدة
            order_type = "buy" if new_trend == "bullish" else "sell"
            volume = calculate_lot_size(account_balance=mt5.account_info().balance)
            print(f"Calculated volume: {volume}")

            # استدعاء دالة place_order
            place_order(symbol="XAUUSD", order_type=order_type, volume=volume)

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب نقاط وولف ويف
        wolf_wave_points = calculate_wolf_wave(data)
        calculate_stochastic(data)
        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # استخدام تحليل المشاعر لتعديل استراتيجيتك
        if average_sentiment > 0.1 and open_positions_count < 5:
            place_order("XAUUSD", "buy", volume=0.01)
        elif average_sentiment < -0.1 and open_positions_count < 5:
            place_order("XAUUSD", "sell", volume=0.01)

        # تأمين الصفقة إذا كانت هناك خسارة كبيرة
        if check_open_positions("XAUUSD"):
            hedge_position_with_retrace_limit("XAUUSD")  # تأمين الصفقة
            set_break_even("XAUUSD")  # تعيين بريك إيفن

            # إغلاق الصفقات الرابحة إذا كانت مجموع الأرباح تعادل ثلاث أرباع مجموع الخسائر
            positions = mt5.positions_get(symbol="XAUUSD")
            total_profit = sum(position.profit for position in positions if position.profit > 0)
            total_loss = sum(-position.profit for position in positions if position.profit < 0)

            # تحقق من الشرط
            if total_profit >= (3 / 4) * total_loss and total_loss > 0:  # تأكد من وجود خسائر
                for position in positions:
                    if position.profit > 0:  # إغلاق الصفقة الرابحة
                        request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "position": position.ticket,
                            "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                            "volume": position.volume,
                            "price": mt5.symbol_info_tick("XAUUSD").bid if position.type == 0 else mt5.symbol_info_tick(
                                "XAUUSD").ask,
                            "deviation": 10,
                            "magic": 234000,
                            "comment": "Closing profitable position",
                            "type_time": mt5.ORDER_TIME_GTC,
                            "type_filling": mt5.ORDER_FILLING_IOC,
                        }
                        result = mt5.order_send(request)
                        if result.retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"تم إغلاق الصفقة الرابحة لـ {position.ticket}.")

        # إنشاء ميزات إضافية
        data = create_features(data)  # إنشاء ميزات إضافية
        data = calculate_price_channel(data)  # حساب قناة السعر
        data = advanced_risk_management(data)  # إدارة المخاطر المتقدمة

        # حساب مناطق الأوامر
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على التنبؤ
        last_row = data.iloc[-1]
        predicted_price = model_xgb.predict(data.drop(['target'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        if open_positions_count < 5:  # تحقق من عدم وجود صفقات مفتوحة
            if predicted_price > last_row['close']:
                place_order("XAUUSD", "buy", volume=0.01)
            elif predicted_price < last_row['close']:
                place_order("XAUUSD", "sell", volume=0.01)

        # رسم مناطق الطلب والعرض
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)
        data.dropna(inplace=True)

        # حساب جميع المؤشرات
        data = calculate_rsi(data)
        data = calculate_moving_average(data)
        data = calculate_momentum(data)
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # تحقق من عدد الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol="XAUUSD")
        open_positions_count = len(open_positions)

        # إذا كان عدد الصفقات المفتوحة 7، لا تفتح أي صفقات جديدة
        if open_positions_count >= 7:
            print("عدد الصفقات المفتوحة بلغ الحد الأقصى (7). لا يمكن فتح صفقات جديدة.")
            return

        # تحديد الاتجاه العام باستخدام المتوسط المتحرك
        current_ma = data['MA'].iloc[-1]
        previous_ma = data['MA'].iloc[-2]
        new_trend = "bullish" if current_ma > previous_ma else "bearish"

        # فتح صفقة جديدة بناءً على الاتجاه الجديد إذا كان العدد أقل من 7
        if open_positions_count < 7:
            order_type = "buy" if new_trend == "bullish" else "sell"

            # تحقق إذا كان عدد الصفقات أقل من 4 لفتح صفقات جديدة
            if open_positions_count < 4:
                volume = calculate_lot_size(account_balance=mt5.account_info().balance)
                print(f"Calculated volume: {volume}")
                place_order(symbol="XAUUSD", order_type=order_type, volume=volume)

        # باقي الكود...

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_momentum(data)  # حساب الزخم
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # تحقق من عدد الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol="XAUUSD")
        open_positions_count = len(open_positions)

        # إذا كان عدد الصفقات المفتوحة 7، لا تفتح أي صفقات جديدة
        if open_positions_count >= 7:
            print("عدد الصفقات المفتوحة بلغ الحد الأقصى (7). لا يمكن فتح صفقات جديدة.")
            return

        # تحديد الاتجاه العام باستخدام المتوسط المتحرك
        current_ma = data['MA'].iloc[-1]
        previous_ma = data['MA'].iloc[-2]

        # تحقق مما إذا كان الاتجاه قد تغير
        new_trend = "bullish" if current_ma > previous_ma else "bearish"

        # إغلاق الصفقات الحالية إذا كان الاتجاه قد تغير
        if (new_trend == "bullish" and open_positions and open_positions[0].type == 1) or (
                new_trend == "bearish" and open_positions and open_positions[0].type == 0):
            closed_positions_count = 0  # عداد الصفقات المغلقة

            for position in open_positions:
                request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "position": position.ticket,
                    "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                    "volume": position.volume,
                    "price": mt5.symbol_info_tick("XAUUSD").bid if position.type == 0 else mt5.symbol_info_tick(
                        "XAUUSD").ask,
                    "deviation": 10,
                    "magic": 234000,
                    "comment": "Closing position..."
                }
                result = mt5.order_send(request)

                if result is None or (hasattr(result, 'retcode') and result.retcode != mt5.TRADE_RETCODE_DONE):
                    print(f"فشل في إغلاق الصفقة: {result.retcode if result else 'Unknown error'}")
                else:
                    print(f"تم إغلاق الصفقة بنجاح: {position.ticket}")
                    closed_positions_count += 1  # زيادة عداد الصفقات المغلقة

                # تحقق مما إذا تم إغلاق 3 صفقات
                if closed_positions_count >= 3:
                    break  # الخروج من الحلقة بعد إغلاق 3 صفقات

        # فتح صفقة جديدة بناءً على الاتجاه الجديد
        if open_positions_count < 7:  # تحقق من العدد قبل فتح صفقة جديدة
            order_type = "buy" if new_trend == "bullish" else "sell"
            volume = calculate_lot_size(account_balance=mt5.account_info().balance)
            print(f"Calculated volume: {volume}")

            # استدعاء دالة place_order
            place_order(symbol="XAUUSD", order_type=order_type, volume=volume)

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب نقاط وولف ويف
        wolf_wave_points = calculate_wolf_wave(data)
        calculate_stochastic(data)

        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # استخدام تحليل المشاعر لتعديل استراتيجيتك
        if average_sentiment > 0.1 and open_positions_count < 7:
            place_order("XAUUSD", "buy", volume=0.01)
        elif average_sentiment < -0.1 and open_positions_count < 7:
            place_order("XAUUSD", "sell", volume=0.01)

        # تأمين الصفقة إذا كانت هناك خسارة كبيرة
        if check_open_positions("XAUUSD"):
            hedge_position_with_retrace_limit("XAUUSD")  # تأمين الصفقة
            set_break_even("XAUUSD")  # تعيين بريك إيفن

            # إغلاق الصفقات الرابحة إذا كانت مجموع الأرباح تعادل ثلاث أرباع مجموع الخسائر
            positions = mt5.positions_get(symbol="XAUUSD")
            total_profit = sum(position.profit for position in positions if position.profit > 0)
            total_loss = sum(-position.profit for position in positions if position.profit < 0)

            # تحقق من الشرط
            if total_profit >= (3 / 4) * total_loss and total_loss > 0:  # تأكد من وجود خسائر
                for position in positions:
                    if position.profit > 0:  # إغلاق الصفقة الرابحة
                        request = {
                            "action": mt5.TRADE_ACTION_DEAL,
                            "position": position.ticket,
                            "type": 1 if position.type == 0 else 0,  # عكس نوع الصفقة
                            "volume": position.volume,
                            "price": mt5.symbol_info_tick("XAUUSD").bid if position.type == 0 else mt5.symbol_info_tick(
                                "XAUUSD").ask,
                            "deviation": 10,
                            "magic": 234000,
                            "comment": "Closing profitable position",
                            "type_time": mt5.ORDER_TIME_GTC,
                            "type_filling": mt5.ORDER_FILLING_IOC,
                        }
                        result = mt5.order_send(request)
                        if result.retcode == mt5.TRADE_RETCODE_DONE:
                            print(f"تم إغلاق الصفقة الرابحة لـ {position.ticket}.")

        # إنشاء ميزات إضافية
        data = create_features(data)  # إنشاء ميزات إضافية
        data = calculate_price_channel(data)  # حساب قناة السعر
        data = advanced_risk_management(data)  # إدارة المخاطر المتقدمة

        # حساب مناطق الأوامر
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على التنبؤ
        last_row = data.iloc[-1]
        predicted_price = model_xgb.predict(data.drop(['target'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        if open_positions_count < 7:  # تحقق من عدم وجود صفقات مفتوحة
            if predicted_price > last_row['close']:
                place_order("XAUUSD", "buy", volume=0.01)
            elif predicted_price < last_row['close']:
                place_order("XAUUSD", "sell", volume=0.01)

        # رسم مناطق الطلب والعرض
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
import os
from bs4 import BeautifulSoup
from textblob import TextBlob
from sklearn.model_selection import TimeSeriesSplit
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import time
import logging
from numba import njit
import dotenv

# تحميل المتغيرات السرية من ملف .env
dotenv.load_dotenv()

# إعداد نظام التسجيل (Logging)
logging.basicConfig(
    filename='trading_strategy.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# تهيئة اتصال MT5 بأمان
def initialize_mt5():
    account_number = int(os.getenv('ACCOUNT_NUMBER'))
    password = os.getenv('PASSWORD')
    server = os.getenv('SERVER')

    if not mt5.initialize():
        logging.error("فشل في تهيئة MT5")
        return False

    if not mt5.login(account_number, password, server):
        logging.error("فشل تسجيل الدخول إلى MT5")
        mt5.shutdown()
        return False

    logging.info("تم الاتصال بـ MT5 بنجاح")
    return True

# تحسين أداء الحسابات باستخدام Numba
@njit
def calculate_technical_indicators_numba(close_prices, period=14):
    """حساب المؤشرات الفنية المعجلة باستخدام Numba"""
    delta = np.zeros_like(close_prices)
    for i in range(1, len(close_prices)):
        delta[i] = close_prices[i] - close_prices[i-1]

    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)

    avg_gain = np.zeros_like(close_prices)
    avg_loss = np.zeros_like(close_prices)

    avg_gain[period] = np.mean(gain[1:period+1])
    avg_loss[period] = np.mean(loss[1:period+1])

    for i in range(period+1, len(close_prices)):
        avg_gain[i] = (avg_gain[i-1] * (period-1) + gain[i]) / period
        avg_loss[i] = (avg_loss[i-1] * (period-1) + loss[i]) / period

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# الحصول على البيانات وتحسينها
def get_enhanced_market_data(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_M15, num_bars=2000):
    """جلب بيانات السوق مع التحقق من الجودة"""
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        if rates is None:
            logging.warning("فشل في الحصول على البيانات من MT5")
            return None

        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)

        # التحقق من تكامل البيانات
        time_diff = df.index.to_series().diff().dropna()
        expected_interval = pd.Timedelta(minutes=15)
        if (time_diff != expected_interval).any():
            logging.warning("تحذير: يوجد فجوات في البيانات الزمنية")

        # حساب المؤشرات المعجلة
        close_prices = df['close'].to_numpy()
        df['RSI'] = calculate_technical_indicators_numba(close_prices)

        return df
    except Exception as e:
        logging.error(f"خطأ في جلب البيانات: {str(e)}")
        return None

# إدارة المخاطر المحسنة
class RiskManager:
    def __init__(self, max_risk=0.02, max_positions=5):
        self.max_risk = max_risk
        self.max_positions = max_positions

    def calculate_dynamic_lot_size(self, symbol):
        """حساب حجم العقد الديناميكي مع مراعاة التقلبات"""
        try:
            symbol_info = mt5.symbol_info(symbol)
            if symbol_info is None:
                logging.error(f"معلومات الرمز غير متوفرة: {symbol}")
                return 0.01

            account_info = mt5.account_info()
            if account_info is None:
                logging.error("فشل في الحصول على معلومات الحساب")
                return 0.01

            # حساب قيمة النقطة الديناميكية
            point_value = 1 / 10 ** symbol_info.digits
            pip_value = point_value * (10 if "JPY" in symbol else 1)

            # حساب التقلبات باستخدام ATR
            data = get_enhanced_market_data(symbol)
            atr = self.calculate_atr(data)
            if atr is None:
                return 0.01

            risk_amount = account_info.balance * self.max_risk
            lot_size = risk_amount / (atr * pip_value)
            return round(max(0.01, min(lot_size, 100)), 2)

        except Exception as e:
            logging.error(f"خطأ في حساب الحجم: {str(e)}")
            return 0.01

    @staticmethod
    def calculate_atr(data, period=14):
        """حساب ATR مع التعامل مع القيم المفقودة"""
        try:
            high = data['high'].ffill().bfill()
            low = data['low'].ffill().bfill()
            close = data['close'].ffill().bfill()

            tr1 = high - low
            tr2 = abs(high - close.shift(1))
            tr3 = abs(low - close.shift(1))

            tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
            return tr.rolling(period).mean().iloc[-1]
        except Exception as e:
            logging.error(f"خطأ في حساب ATR: {str(e)}")
            return None

# استراتيجية التداول المتقدمة
class AdvancedTradingStrategy:
    def __init__(self, symbol):
        self.symbol = symbol
        self.risk_manager = RiskManager()
        self.model = self.train_ml_model()

    def train_ml_model(self):
        """تدريب نموذج تعلم الآلة مع التحقق الزمني"""
        data = get_enhanced_market_data(self.symbol)
        if data is None:
            return None

        features = data[['open', 'high', 'low', 'close', 'RSI']]
        target = data['close'].shift(-1).ffill()

        # التقسيم الزمني للبيانات
        tscv = TimeSeriesSplit(n_splits=5)
        model = XGBRegressor(objective='reg:squarederror')

        for train_index, test_index in tscv.split(features):
            X_train, X_test = features.iloc[train_index], features.iloc[test_index]
            y_train, y_test = target.iloc[train_index], target.iloc[test_index]
            model.fit(X_train, y_train)

        return model

    def execute_strategy(self):
        """تنفيذ الاستراتيجية الكاملة"""
        try:
            if not initialize_mt5():
                return

            data = get_enhanced_market_data(self.symbol)
            if data is None:
                return

            # التحقق من التقلبات العالية
            current_atr = self.risk_manager.calculate_atr(data)
            if current_atr > 50:  # قيمة ATR مرتفعة
                logging.warning("تقلبات السوق عالية - إيقاف التداول")
                return

            # التنبؤ بالسعر
            latest_data = data.iloc[-1:][['open', 'high', 'low', 'close', 'RSI']]
            predicted_price = self.model.predict(latest_data)[0]
            current_price = data['close'].iloc[-1]

            # تحليل المشاعر
            news = self.get_market_news()
            sentiment = self.analyze_sentiment(news)

            # اتخاذ قرار التداول
            open_positions = mt5.positions_get(symbol=self.symbol)
            if len(open_positions) >= self.risk_manager.max_positions:
                logging.info("وصل عدد الصفقات المفتوحة إلى الحد الأقصى")
                return

            lot_size = self.risk_manager.calculate_dynamic_lot_size(self.symbol)

            if predicted_price > current_price * 1.002 and sentiment > 0.1:
                self.execute_trade("buy", lot_size)
            elif predicted_price < current_price * 0.998 and sentiment < -0.1:
                self.execute_trade("sell", lot_size)

        except Exception as e:
            logging.error(f"خطأ في الاستراتيجية: {str(e)}")

    def execute_trade(self, order_type, volume):
        """تنفيذ صفقة مع إدارة المخاطر"""
        try:
            tick = mt5.symbol_info_tick(self.symbol)
            price = tick.ask if order_type == "buy" else tick.bid

            # حساب وقف الخسارة وجني الأرباح
            atr = self.risk_manager.calculate_atr(get_enhanced_market_data(self.symbol))
            sl = price - (atr * 2) if order_type == "buy" else price + (atr * 2)
            tp = price + (atr * 3) if order_type == "buy" else price - (atr * 3)

            request = {
                "action": mt5.TRADE_ACTION_DEAL,
                "symbol": self.symbol,
                "volume": volume,
                "type": mt5.ORDER_TYPE_BUY if order_type == "buy" else mt5.ORDER_TYPE_SELL,
                "price": price,
                "sl": sl,
                "tp": tp,
                "deviation": 10,
                "magic": 2024,
                "comment": "استراتيجية متقدمة",
                "type_time": mt5.ORDER_TIME_GTC,
                "type_filling": mt5.ORDER_FILLING_IOC,
            }

            result = mt5.order_send(request)
            if result.retcode != mt5.TRADE_RETCODE_DONE:
                logging.error(f"فشل في تنفيذ الصفقة: {result.comment}")
            else:
                logging.info(f"تم تنفيذ الصفقة: {order_type} {volume} لوت")

        except Exception as e:
            logging.error(f"خطأ في تنفيذ الصفقة: {str(e)}")

    @staticmethod
    def get_market_news():
        """جلب الأخبار المالية"""
        try:
            url = "https://www.investing.com/news/commodities-news"
            headers = {'User-Agent': 'Mozilla/5.0'}
            response = requests.get(url, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            return [article.text.strip() for article in soup.select('.articleItem')[:5]]
        except Exception as e:
            logging.error(f"خطأ في جلب الأخبار: {str(e)}")
            return []

    @staticmethod
    def analyze_sentiment(news_items):
        """تحليل المشاعر باستخدام TextBlob"""
        try:
            if not news_items:
                return 0

            total = sum(TextBlob(item).sentiment.polarity for item in news_items)
            return total / len(news_items)
        except Exception as e:
            logging.error(f"خطأ في تحليل المشاعر: {str(e)}")
            return 0

# التنفيذ الرئيسي
if __name__ == "__main__":
    strategy = AdvancedTradingStrategy("XAUUSD")
    while True:
        try:
            strategy.execute_strategy()
            time.sleep(60)  # تنفيذ الاستراتيجية كل دقيقة
        except KeyboardInterrupt:
            logging.info("تم إيقاف البرنامج بواسطة المستخدم")
            mt5.shutdown()
            break
        except Exception as e:
            logging.error(f"خطأ غير متوقع: {str(e)}")
            mt5.shutdown()
            break

In [ ]:
ACCOUNT_NUMBER=رقم_حسابك_هنا
PASSWORD=كلمة_المرور_هنا
SERVER=اسم_السيرفر_هنا

In [ ]:
pip install python-dotenv MetaTrader5 pandas numpy requests beautifulsoup4 textblob scikit-learn xgboost numba

In [ ]:
def set_break_even_all_positions():
    """
    Sets the stop-loss to different levels based on price progress towards the target
    for all positions in the account, without moving the stop-loss if the new one is lower than the old one.
    """
    # Get all positions in the account
    all_positions = mt5.positions_get()

    if all_positions:
        for position in all_positions:
            symbol = position.symbol  # Get the symbol for the current position
            entry_price = position.price_open
            take_profit = position.tp
            current_stop_loss = position.sl

            # Determine the current price based on the order type
            current_price = mt5.symbol_info_tick(symbol).bid if position.type == 0 else mt5.symbol_info_tick(symbol).ask

            # Calculate the distance to take profit (absolute value)
            distance_to_tp = abs(take_profit - entry_price)

            # Calculate targets (adjust for buy/sell)
            fifty_percent_target = entry_price + (0.36 * distance_to_tp) if position.type == 0 else entry_price - (0.36 * distance_to_tp)
            seventy_five_percent_target = entry_price + (0.63 * distance_to_tp) if position.type == 0 else entry_price - (0.63 * distance_to_tp)
            ninety_percent_target = entry_price + (0.84 * distance_to_tp) if position.type == 0 else entry_price - (0.84 * distance_to_tp)

            new_stop_loss = current_stop_loss

            # Update stop-loss based on current price and order type
            if position.type == 0 and current_price >= fifty_percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.12 * distance_to_tp))
            elif position.type == 1 and current_price <= fifty_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.12 * distance_to_tp))

            if position.type == 0 and current_price >= seventy_five_percent_target:  # Buy order
                new_stop_loss = max(new_stop_loss, entry_price + (0.24 * distance_to_tp))
            elif position.type == 1 and current_price <= seventy_five_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.24 * distance_to_tp))

            if position.type == 0 and current_price >= ninety_percent_target:  # Buy Order
                new_stop_loss = max(new_stop_loss, entry_price + (0.48 * distance_to_tp))
            elif position.type == 1 and current_price <= ninety_percent_target:  # Sell order
                new_stop_loss = min(new_stop_loss, entry_price - (0.48 * distance_to_tp))

            # Check if the new stop-loss is different and valid before updating
            if abs(new_stop_loss - current_stop_loss) > mt5.symbol_info(symbol).trade_tick_size:
                update_stop_loss(position, new_stop_loss)

# Call the function to set break-even for all positions
set_break_even_all_positions()


In [ ]:
if __name__ == "__main__":
    while True:
        # جمع البيانات
        data = get_and_clean_gold_prices()

        if data is not None:
            # تحديث وقف الخسارة لجميع الصفقات
            set_break_even_all_positions()

            # بناء الاستراتيجية أو تنفيذ الأوامر
            build_strategy(data)

        # تأخير زمني بين التكرارات
        time.sleep(20)  # يمكنك تعديل الوقت حسب الحاجة


In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_momentum(data)  # حساب الزخم
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # تحقق من عدد الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol="XAUUSD")
        open_positions_count = len(open_positions)

        # إذا كان عدد الصفقات المفتوحة 5، لا تفتح أي صفقات جديدة
        if open_positions_count >= 5:
            print("عدد الصفقات المفتوحة بلغ الحد الأقصى (5). لا يمكن فتح صفقات جديدة.")
            return

        # تحديد الاتجاه العام باستخدام المتوسط المتحرك
        current_ma = data['MA'].iloc[-1]
        previous_ma = data['MA'].iloc[-2]

        # تحقق مما إذا كان الاتجاه قد تغير
        new_trend = "bullish" if current_ma > previous_ma else "bearish"

        # السعر الحالي
        current_price = data['close'].iloc[-1]

        # تحقق من المسافة بين السعر الحالي وأي من خطوط تفعيل الصفقات المفتوحة
        for position in open_positions:
            entry_price = position.price_open
            if abs(current_price - entry_price) < 172:  # تحقق من المسافة
                print("السعر الحالي قريب جدًا من خط تفعيل صفقة مفتوحة، لا يمكن فتح صفقة جديدة.")
                return

        # فتح صفقة جديدة بناءً على الاتجاه الجديد
        order_type = "buy" if new_trend == "bullish" else "sell"
        volume = calculate_lot_size(account_balance=mt5.account_info().balance)
        print(f"Calculated volume: {volume}")

        price = mt5.symbol_info_tick("XAUUSD").ask if order_type == "buy" else mt5.symbol_info_tick("XAUUSD").bid

        # تعيين SL و TP
        sl = price - 10 if order_type == "buy" else price + 10
        tp = price + 20 if order_type == "buy" else price - 20

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": "XAUUSD",
            "volume": volume,
            "type": 0 if order_type == "buy" else 1,  # 0 للشراء و 1 للبيع
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Opening new position..."
        }

        result = mt5.order_send(request)

        if result is None or (hasattr(result, 'retcode') and result.retcode != mt5.TRADE_RETCODE_DONE):
            print(f"فشل في فتح الصفقة: {result.retcode if result else 'Unknown error'}")
        else:
            print(f"تم فتح الصفقة بنجاح: {order_type} {volume} lots من XAUUSD عند {price}")

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب نقاط وولف ويف
        wolf_wave_points = calculate_wolf_wave(data)
        calculate_stochastic(data)
        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # استخدام تحليل المشاعر لتعديل استراتيجيتك
        if average_sentiment > 0.1 and open_positions_count < 5:
            place_order("XAUUSD", "buy", volume=0.01)
        elif average_sentiment < -0.1 and open_positions_count < 5:
            place_order("XAUUSD", "sell", volume=0.01)

        # تأمين الصفقة إذا كانت هناك خسارة كبيرة
        if check_open_positions("XAUUSD"):
            set_break_even_all_positions()  # تعيين بريك إيفن

        # إنشاء ميزات إضافية
        data = create_features(data)  # إنشاء ميزات إضافية
        data = calculate_price_channel(data)  # حساب قناة السعر
        data = advanced_risk_management(data)  # إدارة المخاطر المتقدمة

        # حساب مناطق الأوامر
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على التنبؤ
        last_row = data.iloc[-1]
        predicted_price = model_xgb.predict(data.drop(['target'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        if open_positions_count < 5:  # تحقق من عدم وجود صفقات مفتوحة
            if predicted_price > last_row['close']:
                place_order("XAUUSD", "buy", volume=0.01)
            elif predicted_price < last_row['close']:
                place_order("XAUUSD", "sell", volume=0.01)

        # رسم مناطق الطلب والعرض
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


In [ ]:
def place_order(symbol, order_type, volume, risk_percentage=0.01):
    """Place an order in MetaTrader 5 with dynamic lot size."""
    try:
        account_info = mt5.account_info()
        if account_info is None:
            print("Failed to retrieve account info.")
            return

        account_balance = account_info.balance  # الحصول على رصيد الحساب

        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} not found.")
            return

        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        order_type_mt5 = 0 if order_type == "buy" else 1  # 0 للشراء و 1 للبيع

        # حساب ATR لتحديد SL و TP
        atr = calculate_atr(get_and_clean_gold_prices(symbol))
        atr_value = atr.iloc[-1] if atr is not None else 0

        # تحقق من الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol=symbol)
        if open_positions:
            for position in open_positions:
                entry_price = position.price_open
                # تحقق من المسافة بين السعر الحالي وأي من الصفات المفتوحة
                if abs(price - entry_price) < 0.5 * atr_value:
                    print("السعر الحالي قريب جدًا من صفقة مفتوحة، لا يمكن فتح صفقة جديدة.")
                    return

        sl = price - 2 * atr_value if order_type == "buy" else price + 2 * atr_value
        tp = price + 3 * atr_value if order_type == "buy" else price - 3 * atr_value

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)

        if result is None:
            print("Order send failed. Result is None.")
            print(f"Error: {mt5.last_error()}")
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
        else:
            print(
                f"Order placed successfully: {order_type} {volume} lots of {symbol} at {price} with SL: {sl} and TP: {tp}")

    except Exception as e:
        print(f"An error occurred while placing an order: {e}")


In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_momentum(data)  # حساب الزخم
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب نقاط وولف ويف
        wolf_wave_points = calculate_wolf_wave(data)
        calculate_stochastic(data)

        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # استخدام تحليل المشاعر لتعديل استراتيجيتك
        open_positions_count = len(mt5.positions_get(symbol="XAUUSD"))
        if average_sentiment > 0.1 and open_positions_count < 5:
            place_order("XAUUSD", "buy", volume=0.01)
        elif average_sentiment < -0.1 and open_positions_count < 5:
            place_order("XAUUSD", "sell", volume=0.01)

        # إنشاء ميزات إضافية
        data = create_features(data)  # إنشاء ميزات إضافية
        data = calculate_price_channel(data)  # حساب قناة السعر
        data = advanced_risk_management(data)  # إدارة المخاطر المتقدمة

        # حساب مناطق الأوامر
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على التنبؤ
        last_row = data.iloc[-1]
        predicted_price = model_xgb.predict(data.drop(['target'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        if open_positions_count < 5:  # تحقق من عدم وجود صفقات مفتوحة
            if predicted_price > last_row['close']:
                place_order("XAUUSD", "buy", volume=0.01)
            elif predicted_price < last_row['close']:
                place_order("XAUUSD", "sell", volume=0.01)

        # رسم مناطق الطلب والعرض
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import time

# ... (Existing code for initialization, login, data fetching, ATR, stop loss update, lot size calculation, news fetching, sentiment analysis, etc.)

def calculate_moving_averages(data, short_period=20, long_period=50):
    """Calculate short and long moving averages."""
    data['MA_short'] = data['close'].rolling(window=short_period).mean()
    data['MA_long'] = data['close'].rolling(window=long_period).mean()
    return data

def calculate_rsi(data, period=14):
    """Calculate RSI."""
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    data['RSI'] = 100 - (100 / (1 + rs))
    return data

def calculate_bollinger_bands(data, window=20, num_std_dev=2):
    """Calculate Bollinger Bands."""
    data['Middle_Band'] = data['close'].rolling(window=window).mean()
    data['Std_Dev'] = data['close'].rolling(window=window).std()
    data['Upper_Band'] = data['Middle_Band'] + (data['Std_Dev'] * num_std_dev)
    data['Lower_Band'] = data['Middle_Band'] - (data['Std_Dev'] * num_std_dev)
    return data

def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    """Calculate MACD."""
    data['EMA_short'] = data['close'].ewm(span=short_window, adjust=False).mean()
    data['EMA_long'] = data['close'].ewm(span=long_window, adjust=False).mean()
    data['MACD'] = data['EMA_short'] - data['EMA_long']
    data['Signal_Line'] = data['MACD'].ewm(span=signal_window, adjust=False).mean()
    return data

def calculate_price_channel(data, window=20):
    """Calculate Price Channel."""
    data['Channel_Upper'] = data['high'].rolling(window=window).max()
    data['Channel_Lower'] = data['low'].rolling(window=window).min()
    data['Channel_Mid'] = (data['Channel_Upper'] + data['Channel_Lower']) / 2
    return data


def build_strategy(data):
    """Complete strategy with indicators and XGBoost."""
    try:
        data = calculate_moving_averages(data)
        data = calculate_rsi(data)
        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)
        data = calculate_price_channel(data)

        # ... (Existing code for news sentiment, order blocks, risk management, etc.)

        # Create trading signals based on indicators (example)
        data['Buy_Signal'] = (data['close'] > data['Upper_Band']) & (data['RSI'] < 30) & (data['MACD'] > data['Signal_Line'])
        data['Sell_Signal'] = (data['close'] < data['Lower_Band']) & (data['RSI'] > 70) & (data['MACD'] < data['Signal_Line'])

        # Prepare data for XGBoost (including the new signals)
        data['target'] = data['close'].shift(-1)  # Use next day's close as target
        data.dropna(inplace=True)

        features = data.drop(['target', 'Buy_Signal', 'Sell_Signal'], axis=1).fillna(0)  # Features
        target = data['target'].fillna(0)  # Target

        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        last_row = data.iloc[-1]
        predicted_price = model_xgb.predict(data.drop(['target', 'Buy_Signal', 'Sell_Signal'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        open_positions = mt5.positions_get(symbol="XAUUSD")
        open_positions_count = len(open_positions)


        if open_positions_count < 5:
            if predicted_price > last_row['close'] or last_row['Buy_Signal']:
                place_order("XAUUSD", "buy", volume=0.01)
            elif predicted_price < last_row['close'] or last_row['Sell_Signal']:
                place_order("XAUUSD", "sell", volume=0.01)

        # ... (Rest of your code)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

# ... (Rest of your code for continuous execution)



In [ ]:
def calculate_momentum(data, period=10):
    """حساب الزخم كما في برنامج MetaTrader"""
    try:
        # حساب الزخم كنسبة مئوية للتغيير في السعر
        data['Momentum'] = ((data['close'] - data['close'].shift(periods=period)) / data['close'].shift(periods=period)) * 100

        # طباعة آخر قيمة من الزخم
        print(f"Momentum: {data['Momentum'].iloc[-1]}")

        return data
    except Exception as e:
        print(f"An error occurred while calculating momentum: {e}")
        return data


In [ ]:
def calculate_bollinger_bands(data, window=20, num_std_dev=2):
    """حساب بولينجر باندز."""
    try:
        # حساب المتوسط المتحرك
        data['Middle_Band'] = data['close'].rolling(window=window).mean()

        # حساب الانحراف المعياري
        data['Std_Dev'] = data['close'].rolling(window=window).std()

        # حساب الحد العلوي والحد السفلي
        data['Upper_Band'] = data['Middle_Band'] + (data['Std_Dev'] * num_std_dev)
        data['Lower_Band'] = data['Middle_Band'] - (data['Std_Dev'] * num_std_dev)

        # طباعة القيم الأخيرة
        print("آخر قيم لبولينجر باندز:")
        print(f"الحد العلوي: {data['Upper_Band'].iloc[-1]}")
        print(f"الحد السفلي: {data['Lower_Band'].iloc[-1]}")
        print(f"المتوسط المتحرك: {data['Middle_Band'].iloc[-1]}")

        return data
    except Exception as e:
        print(f"An error occurred while calculating Bollinger Bands: {e}")
        return data


In [ ]:
def calculate_rsi(data, period=14):
    """حساب RSI"""
    try:
        # حساب الفرق بين الأسعار
        delta = data['close'].diff()

        # حساب المكاسب والخسائر
        gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()

        # حساب النسبة RS
        rs = gain / loss

        # حساب RSI
        data['RSI'] = 100 - (100 / (1 + rs))

        # طباعة آخر قيمة من RSI
        print(f"RSI: {data['RSI'].iloc[-1]}")

        return data
    except Exception as e:
        print(f"An error occurred while calculating RSI: {e}")
        return data


In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_momentum(data)  # حساب الزخم
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب نقاط وولف ويف
        wolf_wave_points = calculate_wolf_wave(data)
        calculate_stochastic(data)

        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # الشروط لعدم فتح أوامر الشراء
        last_row = data.iloc[-1]
        if (last_row['close'] < last_row['MA'] and
            last_row['close'] > last_row['Lower_Band'] and
            last_row['close'] < last_row['Middle_Band'] and
            last_row['MACD'] < last_row['Signal_Line'] and
            last_row['RSI'] < 50 and
            last_row['%D'] > last_row['%K'] and
            last_row['Momentum'] < 0.05):
            print("لا يمكن فتح أمر شراء بسبب الشروط المحددة.")
        else:
            open_positions_count = len(mt5.positions_get(symbol="XAUUSD"))
            if average_sentiment > 0.1 and open_positions_count < 5:
                place_order("XAUUSD", "buy", volume=0.01)

        # الشروط لعدم فتح أوامر البيع
        if (last_row['close'] > last_row['MA'] and
            last_row['close'] < last_row['Upper_Band'] and
            last_row['close'] > last_row['Middle_Band'] and
            last_row['MACD'] > last_row['Signal_Line'] and
            last_row['RSI'] > 50 and
            last_row['%D'] < last_row['%K'] and
            last_row['Momentum'] > 0.05):
            print("لا يمكن فتح أمر بيع بسبب الشروط المحددة.")
        else:
            open_positions_count = len(mt5.positions_get(symbol="XAUUSD"))
            if average_sentiment < -0.1 and open_positions_count < 5:
                place_order("XAUUSD", "sell", volume=0.01)

        # إنشاء ميزات إضافية
        data = create_features(data)  # إنشاء ميزات إضافية
        data = calculate_price_channel(data)  # حساب قناة السعر
        data = advanced_risk_management(data)  # إدارة المخاطر المتقدمة

        # حساب مناطق الأوامر
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على التنبؤ
        predicted_price = model_xgb.predict(data.drop(['target'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        if open_positions_count < 5:  # تحقق من عدم وجود صفقات مفتوحة
            if predicted_price > last_row['close']:
                place_order("XAUUSD", "buy", volume=0.01)
            elif predicted_price < last_row['close']:
                place_order("XAUUSD", "sell", volume=0.01)

        # رسم مناطق الطلب والعرض
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


In [ ]:
pip install MetaTrader5
pip install pandas
pip install numpy
pip install requests
pip install beautifulsoup4
pip install textblob
pip install scikit-learn
pip install xgboost


In [ ]:
# تحقق من وجود خمس صفقات مفتوحة وجميعها رابحة
def check_and_close_winning_positions(symbol):
    open_positions = mt5.positions_get(symbol=symbol)
    winning_positions = [pos for pos in open_positions if pos.profit > 0]  # صفقات رابحة

    if len(winning_positions) == 5:  # إذا كان هناك خمس صفقات رابحة
        last_position = winning_positions[-1]  # الحصول على آخر صفقة رابحة
        entry_price = last_position.price_open
        take_profit = last_position.tp
        current_price = mt5.symbol_info_tick(symbol).bid  # السعر الحالي

        # حساب المسافة بين سعر التفعيل وسعر جني الأرباح
        distance_to_tp = abs(take_profit - entry_price)

        # تحقق من وصول السعر إلى 20% من المسافة
        if current_price >= entry_price + (0.20 * distance_to_tp):
            # غلق جميع الصفقات
            for position in open_positions:
                close_request = {
                    "action": mt5.TRADE_ACTION_DEAL,
                    "symbol": position.symbol,
                    "volume": position.volume,
                    "type": 1 if position.type == 0 else 0,  # إذا كانت صفقة شراء، أغلقها كصفقة بيع
                    "price": current_price,
                    "deviation": 10,
                    "magic": position.magic,
                    "comment": "Closing all winning positions",
                    "type_time": mt5.ORDER_TIME_GTC,
                    "type_filling": mt5.ORDER_FILLING_IOC,
                }
                mt5.order_send(close_request)
            print("تم غلق جميع الصفقات الرابحة.")


In [ ]:
# تنفيذ الاستراتيجية بشكل مستمر
if __name__ == "__main__":
    last_memory_cleanup = time.time()  # حفظ وقت آخر تنظيف للذاكرة
    while True:
        # جمع البيانات
        data = get_and_clean_gold_prices()

        if data is not None:
            # تحديث وقف الخسارة لجميع الصفقات
            set_break_even_all_positions()

            # بناء الاستراتيجية أو تنفيذ الأوامر
            build_strategy(data)

            # تحقق من غلق جميع الصفقات الرابحة إذا كان هناك خمس صفقات مفتوحة
            check_and_close_winning_positions("XAUUSD")

        # تحرير الذاكرة كل خمس ساعات
        current_time = time.time()
        if current_time - last_memory_cleanup >= 60 * 60:  # 5 ساعات
            free_memory()
            last_memory_cleanup = current_time  # تحديث وقت آخر تنظيف للذاكرة

        # تأخير زمني بين التكرارات
        time.sleep(5)  # يمكنك تعديل الوقت حسب الحاجة


In [ ]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import time
import gc

# بدء الاتصال مع MT5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

# تسجيل الدخول
account = 83967
password = "NU6-caQp"
server = "RoyalInvestment-Server"

if not mt5.login(account, password, server):
    print("login failed")
    mt5.shutdown()

def free_memory():
    gc.collect()
    print("Memory has been freed.")

def get_and_clean_gold_prices(symbol="XAUUSD", timeframe=mt5.TIMEFRAME_M15, num_bars=1000):
    try:
        rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        df.set_index('time', inplace=True)

        required_columns = ['close', 'high', 'low', 'volume']
        for col in required_columns:
            if col not in df.columns:
                print(f"Warning: '{col}' column is missing from the data.")

        df = df[[col for col in required_columns if col in df.columns]]
        return df
    except Exception as e:
        print(f"An error occurred while fetching gold prices: {e}")
        return None

def calculate_lot_size(account_balance, risk_percentage=0.01, pip_value=10):
    """حساب حجم العقد بناءً على رصيد الحساب ونسبة المخاطرة."""
    risk_amount = account_balance * risk_percentage
    lot_size = risk_amount / pip_value  # يمكنك تعديل pip_value حسب الأداة المالية
    return max(0.01, lot_size)  # تأكد من أن حجم العقد لا يقل عن 0.01

# (تستمر باقي الدوال بنفس الشكل)

def place_order(symbol, order_type, risk_percentage=0.01):
    """Place an order in MetaTrader 5 with dynamic lot size based on account balance."""
    try:
        account_info = mt5.account_info()
        if account_info is None:
            print("Failed to retrieve account info.")
            return

        account_balance = account_info.balance  # الحصول على رصيد الحساب
        volume = calculate_lot_size(account_balance, risk_percentage)  # حساب حجم العقد

        if not mt5.symbol_select(symbol, True):
            print(f"Symbol {symbol} not found.")
            return

        price = mt5.symbol_info_tick(symbol).ask if order_type == "buy" else mt5.symbol_info_tick(symbol).bid
        order_type_mt5 = 0 if order_type == "buy" else 1  # 0 للشراء و 1 للبيع

        # حساب ATR لتحديد SL و TP
        atr = calculate_atr(get_and_clean_gold_prices(symbol))
        atr_value = atr.iloc[-1] if atr is not None else 0

        # تحقق من الصفقات المفتوحة
        open_positions = mt5.positions_get(symbol=symbol)
        if open_positions:
            for position in open_positions:
                entry_price = position.price_open
                if abs(price - entry_price) < 0.5 * atr_value:
                    print("السعر الحالي قريب جدًا من صفقة مفتوحة، لا يمكن فتح صفقة جديدة.")
                    return

        sl = price - 2 * atr_value if order_type == "buy" else price + 2 * atr_value
        tp = price + 3 * atr_value if order_type == "buy" else price - 3 * atr_value

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": volume,
            "type": order_type_mt5,
            "price": price,
            "sl": sl,
            "tp": tp,
            "deviation": 10,
            "magic": 234000,
            "comment": "Python script order",
            "type_time": mt5.ORDER_TIME_GTC,
            "type_filling": mt5.ORDER_FILLING_IOC,
        }

        result = mt5.order_send(request)

        if result is None:
            print("Order send failed. Result is None.")
            print(f"Error: {mt5.last_error()}")
            return

        if result.retcode != mt5.TRADE_RETCODE_DONE:
            print(f"Order failed: {result.retcode}, Error: {mt5.last_error()}")
        else:
            print(f"Order placed successfully: {order_type} {volume} lots of {symbol} at {price} with SL: {sl} and TP: {tp}")

    except Exception as e:
        print(f"An error occurred while placing an order: {e}")

def build_strategy(data):
    try:
        data['target'] = data['close'].shift(-1)
        data.dropna(inplace=True)

        # حساب جميع المؤشرات
        data = calculate_rsi(data)
        data = calculate_moving_average(data)
        data = calculate_momentum(data)
        premium, discount = calculate_premium_discount(data)
        data['Premium'] = premium
        data['Discount'] = discount
        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # الشروط لفتح أوامر الشراء
        last_row = data.iloc[-1]
        if (last_row['close'] > last_row['MA'] and
            last_row['close'] < last_row['Upper_Band'] and
            last_row['close'] > last_row['Middle_Band'] and
            last_row['MACD'] > last_row['Signal_Line'] and
            last_row['RSI'] > 50 and
            last_row['%D'] < last_row['%K'] and
            last_row['Momentum'] > 0.05):
            print("فتح أمر شراء بسبب الشروط المحددة.")
            open_positions_count = len(mt5.positions_get(symbol="XAUUSD"))
            if average_sentiment > 0.1 and open_positions_count < 5:
                place_order("XAUUSD", "buy")

        # الشروط لفتح أوامر البيع
        elif (last_row['close'] < last_row['MA'] and
              last_row['close'] > last_row['Lower_Band'] and
              last_row['close'] < last_row['Middle_Band'] and
              last_row['MACD'] < last_row['Signal_Line'] and
              last_row['RSI'] < 50 and
              last_row['%D'] > last_row['%K'] and
              last_row['Momentum'] < 0.05):
            print("فتح أمر بيع بسبب الشروط المحددة.")
            open_positions_count = len(mt5.positions_get(symbol="XAUUSD"))
            if average_sentiment < -0.1 and open_positions_count < 5:
                place_order("XAUUSD", "sell")

        # إنشاء ميزات إضافية
        data = create_features(data)
        data = calculate_price_channel(data)
        data = advanced_risk_management(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")

# تنفيذ الاستراتيجية بشكل مستمر
if __name__ == "__main__":
    last_memory_cleanup = time.time()
    while True:
        data = get_and_clean_gold_prices()

        if data is not None:
            set_break_even_all_positions()
            build_strategy(data)

        current_time = time.time()
        if current_time - last_memory_cleanup >= 60 * 60:  # 5 ساعات
            free_memory()
            last_memory_cleanup = current_time

        time.sleep(5)


In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات
        data = calculate_rsi(data)  # حساب مؤشر القوة النسبية
        data = calculate_moving_average(data)  # حساب المتوسط المتحرك
        data = calculate_momentum(data)  # حساب الزخم
        premium, discount = calculate_premium_discount(data)  # حساب العلاوة والخصم
        data['Premium'] = premium
        data['Discount'] = discount

        data = calculate_bollinger_bands(data)
        data = calculate_macd(data)

        # حساب مناطق العرض والطلب
        supply_zones, demand_zones = calculate_supply_demand(data)
        support, resistance = calculate_support_resistance(data)
        data['Support'] = support
        data['Resistance'] = resistance

        # حساب مستويات فيبوناتشي
        fib_levels = fibonacci_retracement(data['high'].max(), data['low'].min())
        for key, value in fib_levels.items():
            data[key] = value

        # حساب نقاط وولف ويف
        wolf_wave_points = calculate_wolf_wave(data)
        calculate_stochastic(data)

        # جلب الأخبار وتحليل المشاعر
        news_titles = fetch_news()
        sentiments = analyze_sentiment(news_titles)
        average_sentiment = np.mean(sentiments) if sentiments else 0
        print(f"Average Sentiment: {average_sentiment}")

        # جلب آخر صف من البيانات
        last_row = data.iloc[-1]

        # حساب عدد الصفقات المفتوحة
        open_positions_count = len(mt5.positions_get(symbol="XAUUSD"))

        # الشروط لفتح أوامر الشراء
        if (last_row['close'] < last_row['MA'] and
            last_row['close'] > last_row['Lower_Band'] and
            last_row['close'] < last_row['Middle_Band'] and
            last_row['MACD'] < last_row['Signal_Line'] and
            last_row['RSI'] < 50 and
            last_row['%D'] > last_row['%K'] and
            last_row['Momentum'] < 0.05):
            if average_sentiment < -0.1 and open_positions_count < 5:
                place_order("XAUUSD", "sell", volume=0.01)
                print("تم فتح أمر بيع.")

        # الشروط لفتح أوامر البيع
        elif (last_row['close'] > last_row['MA'] and
              last_row['close'] < last_row['Upper_Band'] and
              last_row['close'] > last_row['Middle_Band'] and
              last_row['MACD'] > last_row['Signal_Line'] and
              last_row['RSI'] > 50 and
              last_row['%D'] < last_row['%K'] and
              last_row['Momentum'] > 0.05):
            if average_sentiment > 0.1 and open_positions_count < 5:
                place_order("XAUUSD", "buy", volume=0.01)
                print("تم فتح أمر شراء.")

        # إنشاء ميزات إضافية
        data = create_features(data)  # إنشاء ميزات إضافية
        data = calculate_price_channel(data)  # حساب قناة السعر
        data = advanced_risk_management(data)  # إدارة المخاطر المتقدمة

        # حساب مناطق الأوامر
        bullish_blocks, bearish_blocks = calculate_order_blocks(data)

        # إعداد الميزات للمدخلات في نموذج التعلم الآلي
        features = data.drop(['target'], axis=1).fillna(0)  # الميزات
        target = data['target'].fillna(0)  # الهدف

        # تقسيم البيانات
        X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

        # تدريب النموذج
        model_xgb = XGBRegressor()
        model_xgb.fit(X_train, y_train)

        # التنبؤ
        predictions = model_xgb.predict(X_test)
        rmse = np.sqrt(mean_squared_error(y_test, predictions))
        print(f'Root Mean Squared Error: {rmse}')

        # تحديد أوامر الشراء والبيع بناءً على التنبؤ
        predicted_price = model_xgb.predict(data.drop(['target'], axis=1).iloc[-1:].fillna(0))[0]

        print(f"Predicted Price: {predicted_price}, Current Price: {last_row['close']}")

        # فتح أوامر بناءً على التنبؤ فقط إذا كانت الشروط متحققة
        if open_positions_count < 5:  # تحقق من عدم وجود صفقات مفتوحة
            if predicted_price > last_row['close']:
                place_order("XAUUSD", "buy", volume=0.01)
                print("تم فتح أمر شراء بناءً على التنبؤ.")
            elif predicted_price < last_row['close']:
                place_order("XAUUSD", "sell", volume=0.01)
                print("تم فتح أمر بيع بناءً على التنبؤ.")

        # رسم مناطق الطلب والعرض
        احسب_وارسم_مناطق_طلبات_الشراء_والبيع(data)

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


In [ ]:
def get_and_clean_gold_prices(symbol="XAUUSD", timeframes=[mt5.TIMEFRAME_M15, mt5.TIMEFRAME_M30, mt5.TIMEFRAME_H1], num_bars=1000):
    """Reads gold prices from MetaTrader 5 for multiple timeframes, cleans the data, and returns it."""
    all_data = []

    for timeframe in timeframes:
        try:
            rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, num_bars)
            df = pd.DataFrame(rates)
            df['time'] = pd.to_datetime(df['time'], unit='s')
            df.set_index('time', inplace=True)

            required_columns = ['close', 'high', 'low', 'volume']
            for col in required_columns:
                if col not in df.columns:
                    print(f"Warning: '{col}' column is missing from the data.")

            df = df[[col for col in required_columns if col in df.columns]]
            df['timeframe'] = timeframe  # إضافة عمود للإطار الزمني
            all_data.append(df)
        except Exception as e:
            print(f"An error occurred while fetching gold prices for timeframe {timeframe}: {e}")

    # دمج جميع البيانات في DataFrame واحد
    if all_data:
        combined_df = pd.concat(all_data)
        combined_df.sort_index(inplace=True)  # ترتيب البيانات حسب الوقت
        return combined_df
    return None


In [ ]:
def build_strategy(data):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات باستخدام البيانات المجمعة
        data = calculate_rsi(data)
        data = calculate_moving_average(data)
        data = calculate_macd(data)
        data = calculate_stochastic(data)
        data = calculate_bollinger_bands(data)
        data = calculate_momentum(data)  # تأكد من أنك قمت بحساب المومنتوم
        # تابع استخدام المؤشرات الأخرى...

        # الحصول على آخر صف من البيانات
        last_row = data.iloc[-1]

        # الشروط لفتح أوامر الشراء
        if (last_row['close'] > last_row['MA_15M'] and
            last_row['close'] > last_row['MA_30M'] and
            last_row['close'] > last_row['MA_H1'] and
            last_row['MACD_15M'] < last_row['Signal_Line_15M'] and
            last_row['MACD_30M'] < last_row['Signal_Line_30M'] and
            last_row['MACD_H1'] < last_row['Signal_Line_H1'] and
            last_row['RSI_15M'] < 30 and
            last_row['RSI_30M'] < 30 and
            last_row['RSI_H1'] < 30 and
            last_row['Stochastic_15M_K'] < 20 and
            last_row['Stochastic_30M_K'] < 20 and
            last_row['Stochastic_H1_K'] < 20 and
            last_row['close'] < last_row['Bollinger_Lower_15M'] and
            last_row['close'] < last_row['Bollinger_Lower_30M'] and
            last_row['close'] < last_row['Bollinger_Lower_H1'] and
            last_row['Momentum_15M'] > 0 and  # المومنتوم أكبر من 0
            last_row['Momentum_30M'] > 0 and
            last_row['Momentum_H1'] > 0):
            place_order("XAUUSD", "buy", volume=0.01)
            print("تم فتح أمر شراء.")

        # الشروط لفتح أوامر البيع
        elif (last_row['close'] < last_row['MA_15M'] and
              last_row['close'] < last_row['MA_30M'] and
              last_row['close'] < last_row['MA_H1'] and
              last_row['MACD_15M'] > last_row['Signal_Line_15M'] and
              last_row['MACD_30M'] > last_row['Signal_Line_30M'] and
              last_row['MACD_H1'] > last_row['Signal_Line_H1'] and
              last_row['RSI_15M'] > 70 and
              last_row['RSI_30M'] > 70 and
              last_row['RSI_H1'] > 70 and
              last_row['Stochastic_15M_K'] > 80 and
              last_row['Stochastic_30M_K'] > 80 and
              last_row['Stochastic_H1_K'] > 80 and
              last_row['close'] > last_row['Bollinger_Upper_15M'] and
              last_row['close'] > last_row['Bollinger_Upper_30M'] and
              last_row['close'] > last_row['Bollinger_Upper_H1'] and
              last_row['Momentum_15M'] < 0 and  # المومنتوم أقل من 0
              last_row['Momentum_30M'] < 0 and
              last_row['Momentum_H1'] < 0):
            place_order("XAUUSD", "sell", volume=0.01)
            print("تم فتح أمر بيع.")

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


In [ ]:
def build_strategy(data, open_trades):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات."""
    try:
        # تحقق من عدد الصفقات المفتوحة
        if len(open_trades) >= 5:
            print("هناك خمس صفقات مفتوحة بالفعل. لن يتم فتح أي صفقة جديدة.")
            return  # عدم فتح صفقات جديدة

        # إعداد البيانات
        data['target'] = data['close'].shift(-1)  # استخدام سعر الإغلاق في اليوم التالي كهدف
        data.dropna(inplace=True)  # إزالة الصفوف التي تحتوي على قيم NaN

        # حساب جميع المؤشرات باستخدام البيانات المجمعة
        data = calculate_rsi(data)
        data = calculate_moving_average(data)
        data = calculate_macd(data)
        data = calculate_stochastic(data)
        data = calculate_bollinger_bands(data)
        data = calculate_momentum(data)  # تأكد من أنك قمت بحساب المومنتوم
        # تابع استخدام المؤشرات الأخرى...

        # الحصول على آخر صف من البيانات
        last_row = data.iloc[-1]

        # الشروط لفتح أوامر الشراء
        if (last_row['close'] > last_row['MA_15M'] and
            last_row['close'] > last_row['MA_30M'] and
            last_row['close'] > last_row['MA_H1'] and
            last_row['MACD_15M'] < last_row['Signal_Line_15M'] and
            last_row['MACD_30M'] < last_row['Signal_Line_30M'] and
            last_row['MACD_H1'] < last_row['Signal_Line_H1'] and
            last_row['RSI_15M'] < 30 and
            last_row['RSI_30M'] < 30 and
            last_row['RSI_H1'] < 30 and
            last_row['Stochastic_15M_K'] < 20 and
            last_row['Stochastic_30M_K'] < 20 and
            last_row['Stochastic_H1_K'] < 20 and
            last_row['close'] < last_row['Bollinger_Lower_15M'] and
            last_row['close'] < last_row['Bollinger_Lower_30M'] and
            last_row['close'] < last_row['Bollinger_Lower_H1'] and
            last_row['Momentum_15M'] > 0 and
            last_row['Momentum_30M'] > 0 and
            last_row['Momentum_H1'] > 0):
            place_order("XAUUSD", "buy", volume=0.01)
            print("تم فتح أمر شراء.")

        # الشروط لفتح أوامر البيع
        elif (last_row['close'] < last_row['MA_15M'] and
              last_row['close'] < last_row['MA_30M'] and
              last_row['close'] < last_row['MA_H1'] and
              last_row['MACD_15M'] > last_row['Signal_Line_15M'] and
              last_row['MACD_30M'] > last_row['Signal_Line_30M'] and
              last_row['MACD_H1'] > last_row['Signal_Line_H1'] and
              last_row['RSI_15M'] > 70 and
              last_row['RSI_30M'] > 70 and
              last_row['RSI_H1'] > 70 and
              last_row['Stochastic_15M_K'] > 80 and
              last_row['Stochastic_30M_K'] > 80 and
              last_row['Stochastic_H1_K'] > 80 and
              last_row['close'] > last_row['Bollinger_Upper_15M'] and
              last_row['close'] > last_row['Bollinger_Upper_30M'] and
              last_row['close'] > last_row['Bollinger_Upper_H1'] and
              last_row['Momentum_15M'] < 0 and
              last_row['Momentum_30M'] < 0 and
              last_row['Momentum_H1'] < 0):
            place_order("XAUUSD", "sell", volume=0.01)
            print("تم فتح أمر بيع.")

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")


In [ ]:
def build_strategy(data_15M, data_30M, data_H1, open_trades):
    """بناء الاستراتيجية الكاملة مع تحليل جميع المؤشرات لكل إطار زمني."""
    try:
        # تحقق من عدد الصفقات المفتوحة
        if len(open_trades) >= 5:
            print("هناك خمس صفقات مفتوحة بالفعل. لن يتم فتح أي صفقة جديدة.")
            return  # عدم فتح صفقات جديدة

        # حساب المؤشرات لكل إطار زمني
        data_15M = calculate_rsi(data_15M)
        data_15M = calculate_moving_average(data_15M)
        data_15M = calculate_macd(data_15M)
        data_15M = calculate_stochastic(data_15M)
        data_15M = calculate_bollinger_bands(data_15M)
        data_15M = calculate_momentum(data_15M)

        data_30M = calculate_rsi(data_30M)
        data_30M = calculate_moving_average(data_30M)
        data_30M = calculate_macd(data_30M)
        data_30M = calculate_stochastic(data_30M)
        data_30M = calculate_bollinger_bands(data_30M)
        data_30M = calculate_momentum(data_30M)

        data_H1 = calculate_rsi(data_H1)
        data_H1 = calculate_moving_average(data_H1)
        data_H1 = calculate_macd(data_H1)
        data_H1 = calculate_stochastic(data_H1)
        data_H1 = calculate_bollinger_bands(data_H1)
        data_H1 = calculate_momentum(data_H1)

        # الحصول على آخر صف من البيانات لكل إطار زمني
        last_row_15M = data_15M.iloc[-1]
        last_row_30M = data_30M.iloc[-1]
        last_row_H1 = data_H1.iloc[-1]

        # الشروط لفتح أوامر الشراء
        if (last_row_15M['close'] > last_row_15M['MA_15M'] and
            last_row_30M['close'] > last_row_30M['MA_30M'] and
            last_row_H1['close'] > last_row_H1['MA_H1'] and
            last_row_15M['MACD'] < last_row_15M['Signal_Line'] and
            last_row_30M['MACD'] < last_row_30M['Signal_Line'] and
            last_row_H1['MACD'] < last_row_H1['Signal_Line'] and
            last_row_15M['RSI_14'] < 30 and
            last_row_30M['RSI_14'] < 30 and
            last_row_H1['RSI_14'] < 30 and
            last_row_15M['%K'] < 20 and
            last_row_30M['%K'] < 20 and
            last_row_H1['%K'] < 20 and
            last_row_15M['close'] < last_row_15M['BB_Lower'] and
            last_row_30M['close'] < last_row_30M['BB_Lower'] and
            last_row_H1['close'] < last_row_H1['BB_Lower'] and
            last_row_15M['Momentum_14M'] > 0 and
            last_row_30M['Momentum_14M'] > 0 and
            last_row_H1['Momentum_14M'] > 0):
            place_order("XAUUSD", "buy", volume=0.01)
            print("تم فتح أمر شراء.")

        # الشروط لفتح أوامر البيع
        elif (last_row_15M['close'] < last_row_15M['MA_15M'] and
              last_row_30M['close'] < last_row_30M['MA_30M'] and
              last_row_H1['close'] < last_row_H1['MA_H1'] and
              last_row_15M['MACD'] > last_row_15M['Signal_Line'] and
              last_row_30M['MACD'] > last_row_30M['Signal_Line'] and
              last_row_H1['MACD'] > last_row_H1['Signal_Line'] and
              last_row_15M['RSI_14'] > 70 and
              last_row_30M['RSI_14'] > 70 and
              last_row_H1['RSI_14'] > 70 and
              last_row_15M['%K'] > 80 and
              last_row_30M['%K'] > 80 and
              last_row_H1['%K'] > 80 and
              last_row_15M['close'] > last_row_15M['BB_Upper'] and
              last_row_30M['close'] > last_row_30M['BB_Upper'] and
              last_row_H1['close'] > last_row_H1['BB_Upper'] and
              last_row_15M['Momentum_14M'] < 0 and
              last_row_30M['Momentum_14M'] < 0 and
              last_row_H1['Momentum_14M'] < 0):
            place_order("XAUUSD", "sell", volume=0.01)
            print("تم فتح أمر بيع.")

    except Exception as e:
        print(f"An error occurred while building the strategy: {e}")
